# Add Methane into a Cluster
Take a water in the middle and make it methane!

In [1]:
from ase.io import read
from ase.db import connect
from ase.build import molecule
from scipy.optimize import minimize
from random import shuffle, seed
from tqdm import tqdm
import numpy as np

Configuration

In [2]:
number_to_make = 1024

## Load in Some Clusters
Get all of the clusters we used for training

In [3]:
with connect('../initial-database/initial-ttm.db') as db:
    water_clusters = []
    for row in db.select():
        atoms = row.toatoms()
        atoms.info['filename'] = row.filename
        water_clusters.append(atoms)
print(f'Loaded {len(water_clusters)} water clusters')

Loaded 1720 water clusters


Shuffle them

In [4]:
seed(4)
shuffle(water_clusters)

In [5]:
assert water_clusters[0].info['filename'] == 'mctbp_8909.xyz'  # Ensure we get the same order always

## Make a Function to do the 
We're going to take a water in the middle and make it a methane

In [6]:
def replace_inner_water(atoms):
    """Replace the centermost water in a structure with a methane
    
    Args:
        atoms: Structure to alter
    Returns:
        A structure that now includes a water
    """
    
    # Create a copy
    atoms = atoms.copy()
    
    # Find the water closest to the center of mass
    center_O = np.linalg.norm(atoms.positions[::3] - atoms.get_center_of_mass(), axis=1).argmin()

    # Delete that water
    center_O *= 3 
    center_pos = atoms.positions[center_O]
    del atoms[center_O:center_O + 3]  # Next two atoms are its waters
    assert atoms.get_chemical_formula(empirical=True) == 'H2O'
    
    # Combine the structures given a certain rotation angle
    def _combine_with_rotation(x):
        """Rotate methane, combine and then return the resultant structure and bond distances"""        
        # Make a rotated copy of methane
        rot_meth = molecule('CH4')
        rot_meth.euler_rotate(*x)
        
        # Translate it
        rot_meth.set_center_of_mass(center_pos)
        
        # Combine the structures
        new_strc = atoms + rot_meth
        new_strc.info = atoms.info

        # Measure distances
        dists = new_strc.get_all_distances()
        min_dist = dists[-5:,:-5].min()
        
        return new_strc, min_dist
    
    # Find the rotation of the methane that produces the (locally) maximize distances
    res = minimize(lambda x: -_combine_with_rotation(x)[1], [10, 100, 0], method='Nelder-Mead')
    new_strc, min_dist = _combine_with_rotation(res.x)
    assert min_dist > 0.5, f'Distances are within {min_dist:.2f}'
    
    return new_strc, min_dist

In [7]:
replace_inner_water(water_clusters[0])

(Atoms(symbols='OH2OH2OH2OH2OH2OH2OH2CH4', pbc=False), 1.746739658915813)

## Make a bunch of them
Give our algorithms a good place to start

In [ ]:
with connect('methane-added.db', append=False) as db:
    for a in tqdm(water_clusters[:number_to_make]):
        atoms, min_dist = replace_inner_water(a)
        db.write(atoms, **atoms.info)

 96%|█████████▌| 981/1024 [02:29<00:06,  6.47it/s]